# A2: Vector Semantics

Nikolai Ilinykh, Mehdi Ghanimifard, Wafia Adouane and Simon Dobnik

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read [the following instructions](https://github.com/sdobnik/computational-semantics/blob/master/README.md) on how to work on group assignments.

Write all your answers and the code in the appropriate boxes below.

In this lab we will look at how to build distributional semantic models from corpora and use semantic similarity captured by these models to do semantic tasks. We are also going to examine how different vector composition functions for phrases affect both the model and the learned information about similarities.  

Note that this lab uses a code from `dist_erk.py`, which contains functions that highly resemble those shown during the lecture. In the end, you can use either of the functions (from the lecture / from the file) to solve the tasks.

In [1]:
# the following command simply imports all the methods from that code.
from dist_erk import *

## 1. Loading a corpus

To train a distributional model, we first need a sufficiently large collection of texts which contain different words used frequently enough in different contexts. Here we will use a section of the Wikipedia corpus which you can download from [here](https://linux.dobnik.net/cloud/index.php/s/isMBj49jt5renYt?path=%2Fresources%2Fa2-distributional-representations) (wikipedia.txt.zip). (This file has been borrowed from another lab by [Richard Johansson](http://www.cse.chalmers.se/~richajo/)).  
When unpacked, the file is 151mb, hence if you are using the MLT servers you should store it in a temporary folder outside your home and adjust the `corpus_dir` path below.  
<!-- <It may already exist in `/opt/mlt/courses/cl2015/a5`.> -->


In [2]:
corpus_dir = '/scratch/lt2213-vt21/'

## 2. Building a model

Now you are ready to build the model.  
Using the methods from the code imported above build three word matrices with 1000 dimensions as follows:  

(i) with raw counts (saved to a variable `space_1k`);  
(ii) with PPMI (`ppmispace_1k`);  
(iii) with reduced dimensions SVD (`svdspace_1k`).  
For the latter use `svddim=5`. **[5 marks]**

Your task is to replace `...` with function calls. Functions are imported from `dist_erk.py` earlier, and they largely resemble functions shown during the lecture.

In [3]:
numdims = 1000
svddim = 5

# which words to use as targets and context words?
# we need to count the words and keep only the N most frequent ones
# which function would you use here with which variable?
ktw = do_word_count(corpus_dir, numdims)

wi = make_word_index(ktw) # word index
words_in_order = sorted(ktw) # sorted words

# create different spaces (the original matrix space, the ppmi space, the svd space)
# which functions with which arguments would you use here?
print('create count matrices')
space_1k = make_space(corpus_dir, wi, numdims)
print('ppmi transform')
ppmispace_1k = ppmi_transform(space_1k, wi)
print('svd transform')
svdspace_1k = svd_transform(space_1k, numdims, svddim)
print('done.')

reading file wikipedia.txt
create count matrices
reading file wikipedia.txt
ppmi transform
svd transform
done.


In [4]:
# now, to test the space, you can print vector representation for some words
print('house:', space_1k['house'])

house: [2554 3774 3105  567  962  631  443  185  311  189  131   28   93  169
   81  125  151  408  194   90   79   29  217  184   62   15   31   70
   10    1   41   21    1   31   37    1   30    5   25    7    3   20
   11    1   32   36    2    5   66    4    0   46    8   18   28    0
   20    7    8   16   10   40    0  175   10    2    7   19    1  174
   11    3    1    6    0    0    0   10    9   11    7   24    4    4
   14   23   58    7    0   10    2    3   10    6   18    6   13    3
   22    0    3    5    3    7   14    3   40   20   19   15    6    8
   24    4    5    1   19    0    3    1    0   14    0   14   53    7
    7   11    6    5    5    4   12    6   53    1    1  433    4    0
    5    7    7   12    1    1    3    4   17    8   16    1    2   31
    1   12   14    1   44    6   14    9   38    7    2    6    8    1
   10    6   10    1    9    7    9    4    3   10    0   11    3    2
    0    2   11   37    2    0    2    1    5    9   10   16   88    6

Oxford Advanced Dictionary has 185,000 words, hence 1,000 words is not representative. We trained a model with 10,000 words, and 50 dimensions on truncated SVD. It took 40 minutes on a laptop. We saved all three matrices [here](https://linux.dobnik.net/cloud/index.php/s/isMBj49jt5renYt?path=%2Fresources%2Fa2-distributional-representations) (pretrained.zip). Download them and unpack them to a `pretrained` folder which should be a subfolder of the folder with this notebook:

In [5]:
import numpy as np

numdims = 10000
svddim = 50

print('Please wait...')
ktw_10k       = np.load('/scratch/lt2213-vt21/pretrained/ktw_wikipediaktw.npy', allow_pickle=True)
space_10k     = np.load('/scratch/lt2213-vt21/pretrained/raw_wikipediaktw.npy', allow_pickle=True).all()
ppmispace_10k = np.load('/scratch/lt2213-vt21/pretrained/ppmi_wikipediaktw.npy', allow_pickle=True).all()
svdspace_10k  = np.load('/scratch/lt2213-vt21/pretrained/svd50_wikipedia10k.npy', allow_pickle=True).all()
print('Done.')


Please wait...
Done.


In [23]:
# testing semantic space
print('house:', space_10k['house'])

house: [0.         0.77614985 1.02680071 ... 0.         0.         0.        ]


## 3. Testing semantic similarity

The file `similarity_judgements.txt` (a copy is included with this notebook) contains 7,576 pairs of words and their lexical and visual similarities (based on the pictures) collected through crowd-sourcing using Mechanical Turk as described in [1]. The score range from 1 (highly dissimilar) to 5 (highly similar). Note: this is a different dataset from the phrase similarity dataset we discussed during the lecture (the one from [2]). For more information, please read the papers.

The following code will transform similarity scores into a Python-friendly format:

In [7]:
word_pairs = [] # test suit word pairs
semantic_similarity = [] 
visual_similarity = []
test_vocab = set()

for index, line in enumerate(open('similarity_judgements.txt')):
    data = line.strip().split('\t')
    if index > 0 and len(data) == 3:
        w1, w2 = tuple(data[0].split('#'))
        # it will check if both words from each pair exist in the word matrix.
        if w1 in ktw_10k and w2 in ktw_10k:
            word_pairs.append((w1, w2))
            test_vocab.update([w1, w2])
            semantic_similarity.append(float(data[1]))
            visual_similarity.append(float(data[2]))
        
print('number of available words to test:', len(test_vocab-(test_vocab-set(ktw))))
print('number of available word pairs to test:', len(word_pairs))
#list(zip(word_pairs, visual_similarity, semantic_similarity))

number of available words to test: 12
number of available word pairs to test: 774


Now we are going to test how the cosine similarity between vectors of each of the three spaces (normal space, ppmi, svd) compares with the human similarity judgements for the words in the similarity dataset. Which of the three spaces best approximates human judgements?

For comparison of several scores, we can use [Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman's_rank_correlation_coefficient) which is implemented in `scipy.stats.spearmanr` [here](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.stats.spearmanr.html). The values of the Sperman correlation coefficient range from -1, 0 to 1, where 0 indicates no correlation, 1 perfect correaltion and -1 negative correlation. Hence, the greater the number the better the similarity scores align. The p values tells us if the coefficient is statistically significant. For this to be the case, it must be less than or equal to $< 0.05$.

Here is how you can calculate Pearson's correlation coefficient betweeen the scores of visual similarity and semantic similarity of the available words in the test suite:

In [8]:
from scipy import stats

rho, pval = stats.spearmanr(semantic_similarity, visual_similarity)
print("""Visual Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))


Visual Similarity vs. Semantic Similarity:
rho     = 0.7122
p-value = 0.0000


Let's now calculate the cosine similarity scores of all word pairs in an ordered list using all three matrices. **[6 marks]**

In [9]:
raw_similarities  = [cosine(w1, w2, space_10k) for w1, w2 in word_pairs]
ppmi_similarities = [cosine(w1, w2, ppmispace_10k) for w1, w2 in word_pairs]
svd_similarities  = [cosine(w1, w2, svdspace_10k) for w1, w2 in word_pairs]

Now, calculate correlation coefficients between lists of similarity scores and the real semantic similarity scores from the experiment. The scores of what model best correlates them? Is this expected? **[6 marks]**

In [10]:
rho, pval = stats.spearmanr(semantic_similarity, raw_similarities)
print("""Raw Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f} \n""".format(rho, pval))

rho, pval = stats.spearmanr(semantic_similarity, ppmi_similarities)
print("""PPMI Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f} \n""".format(rho, pval))

rho, pval = stats.spearmanr(semantic_similarity, svd_similarities)
print("""SVD Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f} \n""".format(rho, pval))

Raw Similarity vs. Semantic Similarity:
rho     = 0.1522
p-value = 0.0000 

PPMI Similarity vs. Semantic Similarity:
rho     = 0.4547
p-value = 0.0000 

SVD Similarity vs. Semantic Similarity:
rho     = 0.4232
p-value = 0.0000 



Maybe the p-value=0 is not expected... I wanna see numbers! 
PPMI performes the best as it is closer to the human-judged similarity measures. I think this is expected, because although it uses long and sparse vectors, these vectors may generalize better than short vectors that may be too specific to some cases.
Also, SVD's score is very close to PPMI, which also makes sense (... I guess?) because it is known for removing noisy data e.g., uninformative words which also helps generalizing the output.

We can also calculate correlation coefficients between lists of cosine similarity scores and the real visual similarity scores from the experiment. Which similarity model best correlates with them? How do the correlation coefficients compare with those from the previous comparison - and can you speculate why do we get such results? **[7 marks]**

In [11]:
rho, pval = stats.spearmanr(visual_similarity, raw_similarities)
print("""Raw Similarity vs. Visual Similarity:
rho     = {:.4f}
p-value = {:.4f} \n""".format(rho, pval))

rho, pval = stats.spearmanr(visual_similarity, ppmi_similarities)
print("""PPMI Similarity vs. Visual Similarity:
rho     = {:.4f}
p-value = {:.4f} \n""".format(rho, pval))

rho, pval = stats.spearmanr(visual_similarity, svd_similarities)
print("""SVD Similarity vs. Visual Similarity:
rho     = {:.4f}
p-value = {:.4f} \n""".format(rho, pval))

Raw Similarity vs. Visual Similarity:
rho     = 0.1212
p-value = 0.0007 

PPMI Similarity vs. Visual Similarity:
rho     = 0.3838
p-value = 0.0000 

SVD Similarity vs. Visual Similarity:
rho     = 0.3097
p-value = 0.0000 



PPMI wins again wohooo :woman_dancing:
... and a cute '7' in the p-value of Raw vs Visual, which means that the coorelation is statistically more significant than the other simlilarities. 

## 4. Operations on similarities

We can perform mathematical operations on vectors to derive meaning predictions. For example, we can subtract the normalised vectors for `king` minus `queen` and add the resulting vector to `man` and we hope to get the vector for `woman`. Why? **[3 marks]**

https://kawine.github.io/blog/assets/parallelogram.png

Since Man and Woman share the relationship Male-Female, a man can be represented as (PERSON, Male) or (PERSON, -Female).
Similarly, King and Queen share the same Male-Female relationship, so a King is a (PERSON, royal, Male) and a queen is a (PERSON, royal, Female).

Now, if we want to apply the operation:
King - Queen + Man = woman

(PERSON, royal, Male) - (PERSON, royal, Female) + (PERSON, Male)= 

(+PERSON, +royal, +Male) + (-PERSON, -royal, -Female) + (+PERSON, +Male)= 

(+PERSON, +royal, +Male) + (-PERSON, -royal, +Male) + (+PERSON, +Male)= 

Male + Male + PERSON + Male = very masculine man, which is obv very far away from woman ... AAAAAA this doesn't make any sense x_x


### Another example that makes sense:

King - Man + Woman = Queen

(PERSON, royal, Male) -(PERSON, Male) + (PERSON, Female) = 

(+PERSON, +royal, +Male) +(-PERSON, -Male) + (+PERSON, +Female) = 

royal + PERSON + Female = Queen [YES THIS MAKES SENSE!!!!]

Here is some helpful code that allows us to calculate such comparisons.

In [12]:
from scipy.spatial import distance

def normalize(vec):
    return vec / veclen(vec)

def find_similar_to(vec1, space):
    # vector similarity funciton
    #sim_fn = lambda a, b: 1-distance.euclidean(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.correlation(a, b)
    #sim_fn = lambda a, b: 1-distance.cityblock(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.chebyshev(normalize(a), normalize(b))
    #sim_fn = lambda a, b: np.dot(normalize(a), normalize(b))
    sim_fn = lambda a, b: 1-distance.cosine(a, b)

    sims = [
        (word2, sim_fn(vec1, space[word2]))
        for word2 in space.keys()
    ]
    return sorted(sims, key = lambda p:p[1], reverse=True)

Here is how you apply this code. Comment on the results you get. **[3 marks]**

In [13]:
short = normalize(svdspace_10k['short'])
light = normalize(svdspace_10k['light'])
long = normalize(svdspace_10k['long'])
heavy = normalize(svdspace_10k['heavy'])

find_similar_to(light - (heavy - long), svdspace_10k)[:10]

[('long', 0.8733111261346901),
 ('above', 0.8259671977311955),
 ('around', 0.8030776291120685),
 ('sun', 0.7692439111243973),
 ('just', 0.7678481974778111),
 ('wide', 0.767257431992253),
 ('each', 0.7665960260861158),
 ('circle', 0.7647746702909336),
 ('length', 0.7601066921319761),
 ('almost', 0.7542351860536628)]

light - (heavy - long) should result in 'short' since light and heavy cancel out each other and -long remains, which means short.
So the results we get are basically the 10 most similar words to 'short' ... which is confusing, because most of the words are not similar to 'short' ... maybe because light is percieved as a visual light not something that doesn't weigh much??

Find 5 similar pairs of pairs of words and test them. Hint: Google for `word analogies examples`. You can also construct analogies that are less lexical but more grammatical, e.g. `see, saw, leave, ?` or analogies that are based on world knowledge as in the [Google analogy dataset](http://download.tensorflow.org/data/questions-words.txt) from [3]. Does the resulting vector similarity confirm your expectations? But remember you can only do this if the words are contained in our vector space with 10,000 dimensions. **[10 marks]**

In [14]:
up = normalize(svdspace_10k['up'])
down = normalize(svdspace_10k['down'])

sell = normalize(svdspace_10k['sell'])
buy = normalize(svdspace_10k['buy'])

sick = normalize(svdspace_10k['sick'])
healthy = normalize(svdspace_10k['healthy'])

slow = normalize(svdspace_10k['slow'])
fast = normalize(svdspace_10k['fast'])

actor = normalize(svdspace_10k['actor'])
actress = normalize(svdspace_10k['actress'])

print(""" FAST \n
{} \n \n
""".format(find_similar_to(actor - (actress - slow), svdspace_10k)[:10]))
print(""" ACTRESS \n
{} \n \n
""".format(find_similar_to(sell - (buy - actor), svdspace_10k)[:10]))
print(""" SELL \n
{} \n \n
""".format(find_similar_to(healthy - (sick - buy), svdspace_10k)[:10]))
print(""" SICK \n
{} \n \n
""".format(find_similar_to(down - (up - healthy), svdspace_10k)[:10]))
print(""" UP \n
{} \n \n
""".format(find_similar_to(fast - (slow - down), svdspace_10k)[:10]))
#the results make sense. However, one thing to note is that sometimes the word is most similar to itself and some other times it is most similar to its antonym... what does this tell?


 FAST 

[('slow', 0.9558634925759265), ('fast', 0.8632783574348906), ('faster', 0.8103896971279563), ('rapid', 0.7999448691464592), ('slower', 0.7940595280339026), ('steady', 0.7897594297470079), ('quick', 0.7845645191401889), ('safe', 0.7683359734791474), ('pace', 0.7681348746618416), ('relatively', 0.7663484595322351)] 
 


 ACTRESS 

[('actor', 0.9696457692789059), ('comedian', 0.9490025873983022), ('actress', 0.9168844718458111), ('director', 0.8963290354028031), ('musician', 0.8942853645809059), ('singer', 0.893160567509346), ('cartoonist', 0.8828939688359432), ('journalist', 0.8821038259307022), ('cricketer', 0.865652954594571), ('photographer', 0.8638213681861052)] 
 


 SELL 

[('sell', 0.8242686954560456), ('buy', 0.7988752560023821), ('purchase', 0.7874714153983022), ('acquire', 0.7854461986814776), ('produce', 0.7712097058463793), ('maintain', 0.766158810167326), ('obtain', 0.7585145257957805), ('develop', 0.7578372360651898), ('extract', 0.7496942729938608), ('import', 0.73

## 5. Semantic composition and phrase similarity **[20 marks]**

In this task, we are going to look at how different semantic composition models, introduced in [2] correlate with human judgements. The file with the dataset is `mitchell_lapata_acl08.txt` included with this notebook. Your task is to do the following:  

(i) process the dataset, extract pairs of `reference - landmark high` and `reference - landmark low`; you can use the code from the lecture as something to start with. Note that there are 2 landmarks for each reference: one landmark exhibits high similarity with the reference, while another one has low similarity with the reference. A single human participant could have evaluated both of these pairs. For more details, we refer you to the paper.  

(ii) build models of semantic phrase composition: in the lecture we introduced simple additive, simple multiplicative and combined models (details are in [2]). Your task is to take a single pair (a reference or a high similarity landmark or a low similarity landmark) and compute the composition of its vectors using each of these functions. Thus, you will have three compositional models that take a `noun - verb` pair and output a single vector, representing the meaning of this pair. As your semantic space, you can use pretrained spaces (standard space, ppmi or svd) introduced above. It is up to you which space you use, but for someone who runs your code, it should be pretty straightforward to switch between them.

(iii) calculate Spearman correlation between each model's predictions and human judgements; you should have something similar to the scores that are shown in the paper [2]:  

![title](./res.png)

The paper states that they calculated correlations between each individual participant's judgeements and each model's predictions.  

Let's say we have 3 models: simple additive (A), simple multiplicative (M), combined (C).  
From our task dataset, we also know that we have 20 participants.  
Now, for each participant in 20 participants we get all `verb - noun` pairs that these participated evaluated.  
For example:

In [15]:
participant_judgemenets_example = [
 'participant50 chatter child gabble 6 high',
 'participant50 chatter tooth click 2 high',
 'participant50 reel head whirl 5 high',
 'participant50 reel mind stagger 4 low',
 'participant50 reel industry stagger 5 high',
 'participant50 reel man whirl 3 low',
 'participant50 glow fire beam 7 low',
 'participant50 glow face burn 3 low',
 'participant50 glow cigar burn 5 high',
 'participant50 glow skin beam 7 high'
    
]

In [16]:
participant_judgemenets_example

['participant50 chatter child gabble 6 high',
 'participant50 chatter tooth click 2 high',
 'participant50 reel head whirl 5 high',
 'participant50 reel mind stagger 4 low',
 'participant50 reel industry stagger 5 high',
 'participant50 reel man whirl 3 low',
 'participant50 glow fire beam 7 low',
 'participant50 glow face burn 3 low',
 'participant50 glow cigar burn 5 high',
 'participant50 glow skin beam 7 high']

Let's look at the first pair that participant50 evaluated: reference `child chatter` and high-level similarity landmark (as the last word in the row indicates) `child gabble`. The human gave the similarity score of 6 (very similar). Thus, human similarity judgment = [6].  

Our A model's output:  
cosine(p1, p2) = 0.88, where p1 is the result of addition of word vectors in the reference phrase `child gabble`, and p2 is the result of addition of word vectors in the high-level similarity phrase `child chatter`.  

Therefore, we have human rating vector [6] and model A output [0.88]. Next is to compute correlation between these two vectors.

To get an overall score, simply average your correlation scores over all participants, since you are calculating correlation scores per participant.

Of course, your human rating vectors will be longer (e.g., [6, 7, 3, 4, 5]) where each element is a participant's judgement of a specific pair. Each of your models (A, B, C) will produce a single vector of cosine similarity between these same pairs (e.g., [0.89, 0.98, 0.23, 0.65, 0.55]). The goal is to compare each model's cosine similarity vectors with human rating vectors and identify the model which outputs the best result in terms of being the closest to the way human rate similarity between the phrases.

The minimum to do in this task: compute correlations for 3 models mentioned above and human rating for AT LEAST one participant. Elaborate on how different the resulting correlation scores are depending on the model's composition function (additive, multiplicative, combined). For examples on how to interpret the results, look at Section 5 Results of the original paper.

In [22]:
#Preprocessing
# load the task dataset
with open('./mitchell_lapata_acl08.txt', 'r') as f:
    phrase_dataset = f.read().splitlines()
    
# extract pairs of reference - landmark high and reference - landmark low; 
dataset_low = {}
dataset_high = {}

for line in phrase_dataset[1:]:
    participant_id, reference, noun, landmark, rating, hilo = line.split()
        
    reference_phrase = [noun, reference]
    landmark_phrase = [noun, landmark]
    
    # separate the landmark low and high
    if hilo == 'low':
        if participant_id not in dataset_low:
            dataset_low[participant_id] = []
        else:
            dataset_low[participant_id].append((reference_phrase, landmark_phrase, rating, hilo))
        
    elif hilo == 'high':
        if participant_id not in dataset_high:
            dataset_high[participant_id] = []
        else:
            dataset_high[participant_id].append((reference_phrase, landmark_phrase, rating, hilo))



In [ ]:
#reference and landmark are required to compute cosine similarity

cos_res_low = []
cos_res_high = []

vec_rep = ppmispace_1k # the vector representation can be changed here



In [ ]:
#participant id and human score will be required to run correlation analysis (pearson correlation)


**Any comments/thoughts should go here:**

# Literature

  - [1] C. Silberer and M. Lapata. Learning grounded meaning representations with autoencoders. In Proceedings of the 52nd Annual Meeting of the Association for Computational Linguistics, pages 721–732, Baltimore, Maryland, USA, June 23–25 2014 2014. Association for Computational Linguistics.  

  - [2] Mitchell, J., & Lapata, M. (2008). Vector-based Models of Semantic Composition. In Proceedings of ACL-08: HLT (pp. 236–244). Association for Computational Linguistics.
  
  - [3] T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.

## Marks

This assignment has a total of 60 marks.